In [2]:
import pandas as pd

In [3]:
order_dss = pd.read_csv("orders_distance_stores_softmax.csv")
print(order_dss.shape)
order_dss.head()

(136026, 10)


,Unnamed: 0,user_id,store_id,distance,order_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,0,210,0,2.304404,1438665,prior,9,1,18,3.0
1,1,210,0,2.304404,2850206,prior,36,0,7,2.0
2,2,210,0,2.304404,2406913,prior,42,0,8,3.0
3,3,210,0,2.304404,1155933,prior,43,2,23,2.0
4,4,210,0,2.304404,271697,prior,56,5,17,3.0


In [4]:
order_pps = pd.read_csv("order_products__prior_specials.csv")
print(order_pps.shape)
order_pps.head()

(1172312, 6)


,Unnamed: 0,order_id,product_id,add_to_cart_order,reordered,special
0,0,15,19660,1,1,15
1,1,15,21195,2,1,0
2,2,15,7461,3,1,50
3,3,15,2996,4,1,0
4,4,15,32463,5,1,0


In [6]:
df_merge = pd.merge(order_pps,order_dss, on='order_id')
print(df_merge.shape)
df_merge.head()

(1172312, 15)


,Unnamed: 0_x,order_id,product_id,add_to_cart_order,reordered,special,Unnamed: 0_y,user_id,store_id,distance,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,0,15,19660,1,1,15,37332,54901,1,0.304608,prior,51,3,11,2.0
1,1,15,21195,2,1,0,37332,54901,1,0.304608,prior,51,3,11,2.0
2,2,15,7461,3,1,50,37332,54901,1,0.304608,prior,51,3,11,2.0
3,3,15,2996,4,1,0,37332,54901,1,0.304608,prior,51,3,11,2.0
4,4,15,32463,5,1,0,37332,54901,1,0.304608,prior,51,3,11,2.0


In [7]:
df_merge.isna().sum()

Unnamed: 0_x                  0
order_id                      0
product_id                    0
add_to_cart_order             0
reordered                     0
special                       0
Unnamed: 0_y                  0
user_id                       0
store_id                      0
distance                      0
eval_set                      0
order_number                  0
order_dow                     0
order_hour_of_day             0
days_since_prior_order    13871
dtype: int64

In [10]:
df_merge.to_csv('df_merge.csv')

## drop any rows that contain any null values

In [11]:
# to drop any rows that contain any null values
df_merge.dropna(inplace=True)
df_merge.shape

(1158441, 15)

In [12]:
df_merge.isna().sum()

Unnamed: 0_x              0
order_id                  0
product_id                0
add_to_cart_order         0
reordered                 0
special                   0
Unnamed: 0_y              0
user_id                   0
store_id                  0
distance                  0
eval_set                  0
order_number              0
order_dow                 0
order_hour_of_day         0
days_since_prior_order    0
dtype: int64

In [17]:
#df_merge.drop(['Unnamed: 0_x','Unnamed: 0_y'],axis=1, inplace=True)
df_merge.shape

(1158441, 13)

## drop duplicated rows

In [19]:
df_merge.drop_duplicates(keep='first',inplace=True)
df_merge.shape

(1158441, 13)